In [31]:
import requests
import os

In [32]:
os.environ["FIRESTORE_EMULATOR_HOST"] = "127.0.0.1:8080"


In [33]:
from google.cloud import firestore  # pylint: disable=import-error


def get_firestore_db_client():
    """
    returns a firestore database client pointed
    at the correct host
    """
    return firestore.Client()

In [51]:
cities_string = """Beijing	39.9	116.4
Cairo	30.0	31.2
Chicago	41.9	-87.6
Denver	39.7	-105.0
Karachi	24.9	67.0
Lagos	6.5	3.4
Lima	-12.0	-77.0
London	51.5	0.1
Los Angeles	34.1	-118.2
Manila	14.6	121.0
Mexico City	19.4	-99.1
Miami	25.7	-80.2
Moscow	55.7	37.6
Mumbai	19.1	72.8
New York	40.7	-74.0
San Francisco	37.8	-122.4
Sao Paulo	-23.6	-46.6
Sydney	-33.9	151.2
Tehran	35.7	51.3
Toronto	43.7	-79.4"""


def get_cities_dict():
    cities = cities_string.split("\n")
    cities_dict = {}
    for city_row in cities:
        city_split = city_row.split("	")
        city = city_split[0]
        latitude = city_split[1]
        longitude = city_split[-1]
        cities_dict[city] = {
            'lat': latitude,
            'lon': longitude
        }
    return cities_dict


In [52]:
database = {}
cities_dict = get_cities_dict()

In [54]:



def get_city_lat_long(city_name):
    global cities_dict
    return cities_dict[city_name]
    

def upload_raw_response_to_db(city_name, data):
    global database
    db = get_firestore_db_client()
    for d in data:
        if city_name not in database:
            database[city_name] = {}
        database[city_name][d['date']] = d
        # db.collection(f"foo-bar/{city_name}/{d['date']}").document("response").set(d)
        


def get_twin_city(city_name):
    lat_lon = get_city_lat_long(city_name)
    for i in range(2010, 2020):
        meteostate_url = f'https://meteostat.p.rapidapi.com/point/daily?lat={lat_lon['lat']}&lon={lat_lon['lon']}&' + \
                f'start={i}-01-01&end={i+1}-01-01'
        r = requests.get(meteostate_url,
                         headers={
                             "X-RapidAPI-Host":"meteostat.p.rapidapi.com",
                             "X-RapidAPI-Key": "3264402e48msh8b99bdc540b6a68p17240ejsncea4481ccdac"
                         })
        upload_raw_response_to_db(city_name, r.json()['data'])
    

In [61]:
def cache_all_temp_requests():
    global cities_dict
    for city in cities_dict.keys():
        get_twin_city(city)

In [62]:
cache_all_temp_requests()

366
366
367
366
366
366
367
366
366
366
366
366
367
366
366
366
367
366
366
366
366
366
367
366
366
366
367
366
366
366
366
366
367
366
366
366
367
366
366
366
366
366
367
366
366
366
367
366
366
366
366
366
367
366
366
366
367
366
366
366
366
366
367
0
366
366
367
366
366
366
366
366
367
366
366
366
367
366
366
366
366
366
367
366
366
366
367
366
366
366
366
366
367
366
366
366
367
366
366
366
366
366
367
366
366
366
367
366
366
366
366
366
367
366
366
366
367
366
366
366
366
366
367
366
366
366
367
366
366
366
366
366
367
366
366
366
367
366
366
366
366
366
367
366
366
366
367
366
366
366
366
366
367
366
366
366
367
366
366
366
366
366
367
366
366
366
367
366
366
366
366
366
367
366
366
366
367
366
366
366
0
0
0
0
0
0
0
0
0
0
366
366
367
366
366
366
367
366
366
366


In [64]:
database.keys()

dict_keys(['San Francisco', 'Beijing', 'Cairo', 'Chicago', 'Denver', 'Karachi', 'Lagos', 'Lima', 'London', 'Los Angeles', 'Manila', 'Mexico City', 'Miami', 'Moscow', 'Mumbai', 'New York', 'Sao Paulo', 'Sydney', 'Toronto'])

In [ ]:
averages/city_name/{2020-01} 

In [94]:
def get_average_temp_for_month(city, year_month):
    global database
    temp = 0
    count = 0
    for i in range(0, 32):
        day = year_month + "-" + str(i)
        if city not in database:
            return -1000
        if day in database[city] and len(database[city][day]) > 0 and database[city][day]['tavg'] is not None:
            temp += database[city][day]['tavg']
            count += 1
    if count > 0:
        return temp/count
    return -1000


In [95]:
def cache_average_temp_for_city(city):
    global database
    if "average_temp" not in database:
        database['average_temp'] = {}
        
    if city not in database['average_temp']:
        database['average_temp'][city] = {}
        
    for year in range(2010, 2020):
        for month in range(1, 13):
            year_month = f"{year}-{month}"
            if month < 10:
                year_month = f"{year}-0{month}"

            average_temp = get_average_temp_for_month(city, year_month)
            if average_temp != -1000:
                database['average_temp'][city][year_month] = average_temp

In [96]:
def cache_average_temp_for_all_cities():
    for city in cities_dict.keys():
        cache_average_temp_for_city(city)

In [97]:
cache_average_temp_for_all_cities()

In [103]:
import math

In [106]:
abs(-10)

10

In [122]:
def get_twin_city(city, year_month):
    global database
    global cities_dict
    choice = ""
    difference = 1000
    
    average_temp = database["average_temp"][city][year_month]
    for other_city in cities_dict.keys():
        if other_city != city:
            if year_month not in database["average_temp"][other_city]:
                continue
            other_city_temp = database["average_temp"][other_city][year_month]
            print(other_city_temp)
            if other_city_temp is not None:
                new_diff = abs(average_temp - other_city_temp)
                print(new_diff)
                if new_diff < difference:
                    difference = new_diff
                    choice = other_city
    return other_city

In [123]:
# database["average_temp"]["Los Angeles"]

In [124]:
print(get_twin_city("Los Angeles", "2019-03"))

10.036363636363635
5.00909090909091
18.436363636363634
3.390909090909089
4.754545454545455
10.29090909090909
3.9136363636363636
11.131818181818181
26.40909090909091
11.363636363636365
29.5
14.454545454545455
23.05
8.004545454545456
9.854545454545452
5.190909090909093
28.881818181818176
13.83636363636363
18.836363636363636
3.790909090909091
22.872727272727275
7.82727272727273
1.668181818181818
13.377272727272727
28.422727272727276
13.37727272727273
7.55
7.495454545454545
12.945454545454545
2.0999999999999996
21.927272727272726
6.881818181818181
21.990909090909096
6.945454545454551
2.704545454545455
12.34090909090909
Toronto


In [111]:
print(get_twin_city("San Francisco", "2019-03"))
print(get_twin_city("Toronto", "2019-03"))


Toronto
Toronto


In [ ]:
database["average_temp"]["Toronto"][year_month]

In [66]:
get_average_temp_for_month("San Francisco", "2019-03")

12.945454545454545

In [ ]:
curl 'https://meteostat.p.rapidapi.com/point/daily?lat=35.7&lon=51.3&start=2012-01-01&end=2013-01-03' \
    --header 'X-RapidAPI-Host: meteostat.p.rapidapi.com' \
    --header 'X-RapidAPI-Key: 3264402e48msh8b99bdc540b6a68p17240ejsncea4481ccdac'

In [70]:
database["San Francisco"]['2019-03-25']

{'date': '2019-03-25',
 'tavg': 13.5,
 'tmin': 8.9,
 'tmax': 17.8,
 'prcp': 11.9,
 'snow': 0.0,
 'wdir': 184.0,
 'wspd': 16.6,
 'wpgt': None,
 'pres': 1018.5,
 'tsun': None}

In [60]:
len(database['San Francisco'])

3653

In [ ]:
meteostate_url = 
r = requests.get("http://www.example.com/", headers={"Content-Type":"text"})


In [12]:
r = requests.get("https://meteostat.p.rapidapi.com/point/daily?lat=29.90&lon=116.4&start=2010-01-01&end=2011-01-03",
                         headers={
                             "X-RapidAPI-Host":"meteostat.p.rapidapi.com",
                             "X-RapidAPI-Key": "3264402e48msh8b99bdc540b6a68p17240ejsncea4481ccdac"
                         })

[{'date': '2010-01-01',
  'tavg': 3.6,
  'tmin': 0.0,
  'tmax': 6.3,
  'prcp': 0.8,
  'snow': None,
  'wdir': None,
  'wspd': None,
  'wpgt': None,
  'pres': None,
  'tsun': None},
 {'date': '2010-01-02',
  'tavg': 6.3,
  'tmin': 2.9,
  'tmax': 12.3,
  'prcp': 0.0,
  'snow': None,
  'wdir': None,
  'wspd': None,
  'wpgt': None,
  'pres': None,
  'tsun': None},
 {'date': '2010-01-03',
  'tavg': 5.8,
  'tmin': 3.1,
  'tmax': 8.2,
  'prcp': 0.0,
  'snow': None,
  'wdir': None,
  'wspd': None,
  'wpgt': None,
  'pres': None,
  'tsun': None},
 {'date': '2010-01-04',
  'tavg': 4.8,
  'tmin': 2.7,
  'tmax': 7.4,
  'prcp': 0.0,
  'snow': None,
  'wdir': None,
  'wspd': None,
  'wpgt': None,
  'pres': None,
  'tsun': None},
 {'date': '2010-01-05',
  'tavg': -0.2,
  'tmin': -1.6,
  'tmax': 2.7,
  'prcp': 0.0,
  'snow': None,
  'wdir': None,
  'wspd': None,
  'wpgt': None,
  'pres': None,
  'tsun': None},
 {'date': '2010-01-06',
  'tavg': 0.6,
  'tmin': -1.9,
  'tmax': 4.3,
  'prcp': 1.0,
  'snow

In [ ]:
curl 'https://meteostat.p.rapidapi.com/point/daily?lat=29.90&lon=116.4&start=2010-01-01&end=2011-01-03' \
    --header 'X-RapidAPI-Host: meteostat.p.rapidapi.com' \
    --header 'X-RapidAPI-Key: 3264402e48msh8b99bdc540b6a68p17240ejsncea4481ccdac'


In [ ]:
{
  "meta": {
    "generated": "2024-09-23 18:27:36",
    "stations": [
      "58424",
      "58527"
    ]
  },
  "data": [
    {
      "date": "2020-01-01",
      "tavg": 5.1,
      "tmin": -1.4,
      "tmax": 9,
      "prcp": 3,
      "snow": null,
      "wdir": 26,
      "wspd": 6.4,
      "wpgt": null,
      "pres": 1031.5,
      "tsun": null
    },
    {
      "date": "2020-01-02",
      "tavg": 6.4,
      "tmin": 0.8,
      "tmax": 9,
      "prcp": 0,
      "snow": null,
      "wdir": 30,
      "wspd": 10.4,
      "wpgt": null,
      "pres": 1029.2,
      "tsun": null
    },
    {
      "date": "2020-01-03",
      "tavg": 6.8,
      "tmin": 5,
      "tmax": 7.7,
      "prcp": 4.3,
      "snow": null,
      "wdir": 31,
      "wspd": 11.3,
      "wpgt": null,
      "pres": 1028.2,
      "tsun": null
    }
  ]
}